## ***Machine Learning to predict public sentiment from text data.***

Look into twitter text data to predict if the given text has positive or negative
sentiment towards a particular brand. The dataset includes twitter text related to
Apple and Google products with user sentiment ranked between ‘positive’,
‘negative’, ‘neutral’ and ‘no_idea’, sentiments. Create a simpleRNN or LSTM based
classifiers to classify tweets into the four classes. You can avoid
‘emotion_in_tweet_is_directed_at’ column.

In [34]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SimpleRNN
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping


In [35]:
#load the dataset
data = pd.read_csv('/content/judge-1377884607_tweet_product_company.csv',encoding = 'latin-1')

In [36]:
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [37]:
# Remove 'emotion_in_tweet_is_directed_at' column
data = data.drop('emotion_in_tweet_is_directed_at',axis = 1)

In [38]:
data.head()

,tweet_text,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Positive emotion


In [39]:
# Rename the column
data = data.rename(columns={'is_there_an_emotion_directed_at_a_brand_or_product': 'sentiment'})

In [40]:
data.head()

,tweet_text,sentiment
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Positive emotion


In [41]:
import re

In [42]:
def remove_tags(raw_text):
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', str(raw_text))
    return cleaned_text

data['tweet_text'] = data['tweet_text'].apply(remove_tags)

In [43]:
data.head()

,tweet_text,sentiment
0,wesley I have a G iPhone After hrs tweeting a...,Negative emotion
1,jessedee Know about fludapp Awesome iPadiPhon...,Positive emotion
2,swonderlin Can not wait for iPad also They sh...,Positive emotion
3,sxsw I hope this years festival isnt as crashy...,Negative emotion
4,sxtxstate great stuff on Fri SXSW Marissa Maye...,Positive emotion


In [44]:
X = data['tweet_text'].values  # 'text' column contains the tweet text
y = data['sentiment'].values  # 'sentiment' column contains the sentiment labels


In [45]:
from sklearn.preprocessing import LabelEncoder

In [46]:
# Map sentiment labels to numerical values using label encoding
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


In [47]:
# Convert numerical labels to one-hot encoded format
y_onehot = to_categorical(y_encoded, num_classes=4)


In [48]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)


In [49]:
# Tokenize text and convert to sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

In [50]:
# Pad sequences to have the same length
max_sequence_length = max(len(seq) for seq in X_train_sequences)
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)

LSTM MODEL

In [51]:
# Define the model architecture (LSTM)
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 100, input_length=max_sequence_length))
model.add(LSTM(128))
model.add(Dense(4, activation='softmax'))

In [52]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [53]:
early_stopping = EarlyStopping(patience=3, monitor='val_loss')

In [54]:
# Train the model
model.fit(X_train_padded, y_train, validation_data=(X_test_padded, y_test), epochs=10, batch_size=32)


Epoch 1/10
228/228 [==============================] - 23s 91ms/step - loss: 0.9016 - accuracy: 0.5964 - val_loss: 0.8596 - val_accuracy: 0.6146
Epoch 2/10
228/228 [==============================] - 18s 80ms/step - loss: 0.6790 - accuracy: 0.7244 - val_loss: 0.8130 - val_accuracy: 0.6663
Epoch 3/10
228/228 [==============================] - 18s 78ms/step - loss: 0.4754 - accuracy: 0.8115 - val_loss: 0.8942 - val_accuracy: 0.6756
Epoch 4/10
228/228 [==============================] - 18s 80ms/step - loss: 0.3633 - accuracy: 0.8583 - val_loss: 0.9717 - val_accuracy: 0.6647
Epoch 5/10
228/228 [==============================] - 22s 95ms/step - loss: 0.2982 - accuracy: 0.8837 - val_loss: 0.9998 - val_accuracy: 0.6614
Epoch 6/10
228/228 [==============================] - 19s 81ms/step - loss: 0.2518 - accuracy: 0.8976 - val_loss: 1.1345 - val_accuracy: 0.6537
Epoch 7/10
228/228 [==============================] - 18s 80ms/step - loss: 0.2229 - accuracy: 0.9076 - val_loss: 1.1388 - val_accuracy:

In [55]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

57/57 [==============================] - 1s 18ms/step - loss: 1.4779 - accuracy: 0.6641
Test Loss: 1.477863073348999
Test Accuracy: 0.6641011834144592


In [57]:
def predict_sentiment(tweet_text):
    sequence = tokenizer.texts_to_sequences([tweet_text])
    input_data = pad_sequences(sequence, maxlen=max_sequence_length)
    prediction = model.predict(input_data)[0]
    class_labels = ['positive', 'negative', 'neutral', 'no idea']
    predicted_class_index = np.argmax(prediction)
    predicted_class = class_labels[predicted_class_index]
    return predicted_class

In [58]:
input_text = "I have a 3G iPhone. After 3 hrs tweeting, it was dead! I need to upgrade."
prediction = predict_sentiment(input_text)
print(f"The sentiment prediction for '{input_text}' is: {prediction}")

1/1 [==============================] - 0s 481ms/step
The sentiment prediction for 'I have a 3G iPhone. After 3 hrs tweeting, it was dead! I need to upgrade.' is: negative


In [62]:
input_text = "Awesome iPad/iPhone app that you'll likely appreciate for its design."
prediction = predict_sentiment(input_text)
print(f"The sentiment prediction for '{input_text}' is: {prediction}")

1/1 [==============================] - 0s 35ms/step
The sentiment prediction for 'Awesome iPad/iPhone app that you'll likely appreciate for its design.' is: no idea


In [63]:
input_text = "I just noticed DST is coming this weekend. How many iPhone users will be an hour late at SXSW come Sunday morning?"
prediction = predict_sentiment(input_text)
print(f"The sentiment prediction for '{input_text}' is: {prediction}")

1/1 [==============================] - 0s 24ms/step
The sentiment prediction for 'I just noticed DST is coming this weekend. How many iPhone users will be an hour late at SXSW come Sunday morning?' is: negative


In [65]:
input_text = "SXSW is just starting, CTIA is around the corner and googleio is only a hop skip and a jump from there, good time to be an android fan"
prediction = predict_sentiment(input_text)
print(f"The sentiment prediction for '{input_text}' is: {prediction}")

1/1 [==============================] - 0s 26ms/step
The sentiment prediction for 'SXSW is just starting, CTIA is around the corner and googleio is only a hop skip and a jump from there, good time to be an android fan' is: no idea


In [64]:
input_text = "Apple has opened a pop-up store in Austin so the nerds in town for SXSW can get their new iPads."
prediction = predict_sentiment(input_text)
print(f"The sentiment prediction for '{input_text}' is: {prediction}")

1/1 [==============================] - 0s 27ms/step
The sentiment prediction for 'Apple has opened a pop-up store in Austin so the nerds in town for SXSW can get their new iPads.' is: neutral


In [66]:
input_text = "iPhone app makes it easy to connect on all social networks with people you meet."
prediction = predict_sentiment(input_text)
print(f"The sentiment prediction for '{input_text}' is: {prediction}")

1/1 [==============================] - 0s 25ms/step
The sentiment prediction for 'iPhone app makes it easy to connect on all social networks with people you meet.' is: no idea


In [67]:
# Example usage
input_text = "it was a fine day"
prediction = predict_sentiment(input_text)
print(f"The sentiment prediction for '{input_text}' is: {prediction}")

1/1 [==============================] - 0s 25ms/step
The sentiment prediction for 'it was a fine day' is: positive


In [68]:
input_text = "This movie is amazing!"
prediction = predict_sentiment(input_text)
print(f"The sentiment prediction for '{input_text}' is: {prediction}")

1/1 [==============================] - 0s 37ms/step
The sentiment prediction for 'This movie is amazing!' is: no idea


In [69]:
input_text = "it was a horrible day"
prediction = predict_sentiment(input_text)
print(f"The sentiment prediction for '{input_text}' is: {prediction}")

1/1 [==============================] - 0s 38ms/step
The sentiment prediction for 'it was a horrible day' is: negative


The model might not fully capture the sentiments expressed in the text due to variations in language, expressions, or sarcasm.

Different sentiment analysis models may have different levels of accuracy and may perform differently based on the specific dataset and text inputs.So the predictions may vary.